Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

# Aprendizado profundo com Python

## 8.5 Introdução às redes adversárias geradoras

> Introdução às Redes Adversariais Gerativas

GAN, chinês é ~~ gan ~~, errado, Generative Adversarial Network. Como VAE, é usado para aprender o espaço latente da imagem. Isso pode tornar a imagem gerada "estatisticamente" igual à imagem real, ou seja, a imagem gerada é bastante realista. No entanto, ao contrário do VAE, o espaço latente de GAN não pode ser garantido como tendo uma estrutura significativa e não é contínuo.

GAN consiste em duas partes:

-Rede geradora: insira um vetor aleatório (um ponto aleatório no espaço latente) e decodifique-o em uma imagem.
- Rede discriminadora: Insira uma imagem (real ou desenhada pelo gerador) e preveja se a imagem é real ou criada pela rede do gerador. (A rede discriminadora também é chamada de "adversário", adversário)

O objetivo do treinamento do GAN é permitir que a "rede geradora" engane a "rede discriminadora".

A compreensão intuitiva de GAN é uma história muito inspiradora: ou seja, há duas pessoas, um falsificador e um avaliador. Os falsificadores imitam as pinturas do mestre e, em seguida, misturam suas imitações com as originais e as submetem ao avaliador para avaliação, que avalia a autenticidade de cada pintura e vê através de quais são forjadas. O amável avaliador deu feedback ao falsificador e disse-lhe as características do original. Os falsificadores irão, passo a passo, melhorar sua capacidade de falsificação com base nas opiniões do avaliador. Os dois repetiram esse processo incansavelmente: os falsificadores tornaram-se cada vez mais bons em copiar pinturas de mestres e os avaliadores se tornaram cada vez mais bons em encontrar pinturas falsas. No final, os falsificadores criaram um lote de "produtos de imitação" que os avaliadores impõem.


O método de treinamento do GAN é muito especial e seu mínimo otimizado não é fixo. Nossa "descida gradiente" usual é rolar montanha abaixo ao longo do terreno de perda estática, mas cada degrau da montanha durante o treinamento GAN mudará todo o terreno. É um sistema dinâmico e seu processo de otimização é o equilíbrio entre duas forças. Portanto, GAN é difícil de treinar. Para fazer o GAN funcionar normalmente, é necessário muito trabalho de construção de modelo e ajuste de hiperparâmetro.

### Rede Adversarial Gerativa Convolucional Profunda

Vamos tentar implementar o GAN mais simples e simples com Keras. Especificamente, construiremos uma ** rede convolucional profunda de confronto gerador ** (GAN convolucional profunda, DCGAN). O gerador e o discriminador desse tipo de coisa são ambas redes neurais convolucionais profundas.

Vamos treinar DCGAN com imagens da categoria "sapo" no conjunto de dados CIFAR10. Este conjunto de dados contém 50000 imagens RGB 32 × 32, que pertencem a 10 categorias (5000 imagens em cada categoria).

#### Processo de implementação

1. A rede `generator` mapeia um vetor de forma` (latent_dim,) `para uma imagem de forma` (32, 32, 3) `.
2. A rede `discriminator` mapeia a imagem com a forma` (32, 32, 3) `para uma pontuação binária, que é usada para avaliar a probabilidade de a imagem ser verdadeira.
3. A rede `gan` conecta` gerador` e `discriminador` juntos:` gan (x) = discriminador (gerador (x)) `. Esta rede mapeia o vetor latente para o resultado da avaliação do discriminador.
4. Use as amostras de imagens verdadeiras e falsas rotuladas com `" real "` ou `" falsa "` para treinar o discriminador e use o método usual de treinar modelos de classificação de imagem comuns.
5. Para treinar o gerador, use o gradiente de perda do modelo `gan` em relação ao peso do gerador. A cada etapa, o peso do gerador deve ser movido para "tornar o discriminador mais provável de classificar a imagem decodificada pelo gerador como" verdadeira "", ou seja, treinar o gerador para enganar o discriminador.

#### Habilidades práticas

O processo de treinamento e ajuste do GAN é muito difícil. Portanto, precisamos nos lembrar de algumas habilidades práticas resumidas por pessoas anteriores. Essas técnicas geralmente são úteis, mas não se aplicam a todas as situações. Essas coisas não têm base teórica, são todas metafísicas, então escreva a conclusão diretamente, sem explicação:

-Use tanh como a ativação da última camada do gerador em vez de sigmóide.
-Use a distribuição normal (distribuição gaussiana) para amostrar pontos no espaço latente em vez da distribuição uniforme.
-A aleatoriedade pode melhorar a robustez. O treinamento GAN pode ficar "preso" de várias maneiras (atingindo o equilíbrio dinâmico errado). A introdução da aleatoriedade durante o processo de treinamento ajuda a prevenir essa situação. Existem duas maneiras de introduzir a aleatoriedade:
    1. Use dropout no discriminador;
    2. Adicione ruído aleatório ao rótulo do discriminador;
- Gradiente esparso atrapalhará o treinamento GAN "Operação máxima de pooling" e "ativação ReLU" podem levar a gradientes esparsos, por isso é recomendado:
    1. Use `` convolução em etapas '' ao invés de `` max pooling '' para reduzir a resolução;
    2. Use a camada LeakyReLU para substituir a ativação ReLU;
-Nas imagens geradas, artefatos quadriculados são frequentemente vistos, o que se deve à cobertura desigual do espaço do pixel no gerador. A solução para este problema é: sempre que o passo Conv2DTranpose ou Conv2D é usado no gerador e discriminador, o tamanho do kernel deve ser divisível pelo passo.


#### A realização do gerador

Comecei a construir o primeiro GAN para jovens! !

Primeiro desenvolva o modelo do gerador: converta o vetor do espaço latente em uma imagem candidata.

Para evitar "travar" durante o treinamento, o dropout é usado tanto no discriminador quanto no gerador.

In [1]:
# GAN Rede Geradora

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

height = 32
width = 32
channels = 3

latent_dim = 32

generator_input = keras.Input(shape=(latent_dim,))

# Converta a entrada em um mapa de recursos de 128 canais com um tamanho de 16 × 16
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# A amostragem de aumento é de 32 × 32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Gere um mapa de recursos de tamanho 32 × 32 (o formato da imagem CIFAR10)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)      

#### Implementação do discriminador

Em seguida, desenvolva o modelo discriminador, insira uma imagem (real ou sintética) e divida-a em "verdadeiro" (imagem real do conjunto de treinamento) ou "falso" (imagem desenhada pelo gerador).

In [2]:
# GAN Rede discriminadora

discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer,
                      loss='binary_crossentropy')

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)        

#### Confrontando a Internet

Finalmente, configure o GAN, conecte o gerador e o discriminador e converta os pontos no espaço latente em um julgamento de classificação verdadeiro ou falso.

Ao treinar este modelo, é necessário congelar o “discriminador” (torná-lo intratável), e apenas permitir que o “gerador” se mova na direção de “melhorar a habilidade do discriminador de engano”.

Ao treinar gan, tudo o que usamos são os rótulos de "imagens reais", portanto, se o peso do "discriminador" puder ser atualizado durante o processo de treinamento, o treinamento fará com que o discriminador sempre preveja "verdadeiro".

In [3]:
# Rede de Confronto

discriminator.trainable = False    # Isso só funcionará em gan

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(
    lr=0.0004,
    clipvalue=1.0,
    decay=1e-8)
gan.compile(optimizer=gan_optimizer,
            loss='binary_crossentropy')

#### Treine DCGAN

O fluxo do loop de treinamento é o seguinte:

1. Extraia pontos aleatórios (ruído aleatório) do espaço latente.
2. Dê este ruído aleatório ao gerador para gerar uma imagem.
3. Misture a imagem gerada com a imagem real.
4. Use essas imagens misturadas e os rótulos correspondentes (a imagem real é "verdadeira" e a imagem gerada é "falsa") para treinar o discriminador.
5. Desenhe aleatoriamente novos pontos no espaço latente.
6. Use esses vetores e rótulos aleatórios que são "imagens reais" para treinar gan.

Implementação de código específico:

In [26]:
# GAN Treinamento,

import os
import time
from tensorflow.keras.preprocessing import image

# Importar dados CIFAR10
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
# Selecione a imagem do sapo🐸
x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)
).astype('float32') / 255.


iterations = 10000
batch_size = 20
save_dir = 'gan_save'

start = 0
for step in range(iterations+1):
    start_time = time.time()
    
    # Amostra aleatoriamente de pontos potenciais
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # Gerar imagem
    generated_images = generator.predict(random_latent_vectors)
    
    # Escolha uma imagem real
    stop = start + batch_size
    real_images = x_train[start: stop]
    
    # Combine imagens reais e geradas e dê rótulos
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)  # Adicionar ruído aleatório ao rótulo
    
    # Discriminador de treinamento
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # Amostra aleatoriamente de pontos potenciais
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))  # Minta que todas são fotos reais
    
    # Gerador de trem através do modelo gan
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    end_time = time.time()
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 50 == 0:
        gan.save_weights('gan.h5')
        
        print(f'step {step}: discriminator loss: {d_loss}, adversarial loss: {a_loss}')
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, f'generated_frog_{step}.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, f'real_frog_{step}.png'))
    else:
        time_cost = end_time - start_time
        time_eta = time_cost * (iterations - step)
        print(f'{step}/{iterations}: {time_cost:.2f}s - ETA: {time_eta:.0f}s', end='\r')
        

step 0: discriminator loss: 0.6944685578346252, adversarial loss: 0.7566524744033813
step 50: discriminator loss: 0.6997207999229431, adversarial loss: 0.7283448576927185
step 100: discriminator loss: 0.6917451024055481, adversarial loss: 0.7818207144737244
step 150: discriminator loss: 0.7006672620773315, adversarial loss: 0.77472984790802
step 200: discriminator loss: 0.6959202885627747, adversarial loss: 0.7540444731712341
step 250: discriminator loss: 0.6907225847244263, adversarial loss: 0.7319836020469666
step 300: discriminator loss: 0.6964272856712341, adversarial loss: 0.795426070690155
step 350: discriminator loss: 0.7021819949150085, adversarial loss: 0.7412662506103516
step 400: discriminator loss: 0.6954243183135986, adversarial loss: 0.7441832423210144
step 450: discriminator loss: 0.7256754636764526, adversarial loss: 0.7400968074798584
step 500: discriminator loss: 0.7020201683044434, adversarial loss: 0.7446410059928894
step 550: discriminator loss: 0.6801480054855347,

In [27]:
for step in range(iterations, iterations+1):
    start_time = time.time()
    
    # Amostra aleatoriamente de pontos potenciais
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # Gerar imagem
    generated_images = generator.predict(random_latent_vectors)
    
    # Escolha uma imagem real
    stop = start + batch_size
    real_images = x_train[start: stop]
    
    # Combine imagens reais e geradas e dê rótulos
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)  # Adicionar ruído aleatório ao rótulo
    
    # Discriminador de treinamento
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # Amostra aleatoriamente de pontos potenciais
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))  # Minta que todas são fotos reais
    
    # Gerador de trem através do modelo gan
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    end_time = time.time()
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 50 == 0:
        gan.save_weights('gan.h5')
        
        print(f'step {step}: discriminator loss: {d_loss}, adversarial loss: {a_loss}')
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, f'generated_frog_{step}.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, f'real_frog_{step}.png'))
    else:
        time_cost = end_time - start_time
        time_eta = time_cost * (iterations - step)
        print(f'{step}/{iterations}: {time_cost:.2f}s - ETA: {time_eta:.0f}s', end='\r')
        

step 1000: discriminator loss: 0.7096449136734009, adversarial loss: 0.752526581287384


A imagem de saída final:

! [A última imagem de saída] (https://tva1.sinaimg.cn/large/007S8ZIlgy1gi27n2ulptj300w00w0si.jpg)
